In [93]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from parsel import Selector
import pandas as pd
from time import sleep

In [94]:
# navigate to google maps url
url = 'https://www.google.com/maps/place/Central+Park+Zoo/@40.7712318,-73.9674707,15z/data=!3m1!5s0x89c259a1e735d943:0xb63f84c661f84258!4m16!1m8!3m7!1s0x89c258faf553cfad:0x8e9cfc7444d8f876!2sTrump+Tower!8m2!3d40.7624284!4d-73.973794!9m1!1b1!3m6!1s0x89c258f1fcd66869:0x65d72e84d91a3f14!8m2!3d40.767778!4d-73.9718335!9m1!1b1?hl=en&hl=en'


In [95]:
# launch chrome
driver_path = '/Users/genebob/Dropbox/LoL Python v1.0/v2/LoL_Scrape_Bot_v2/chromedriver/chromedriver'
driver = webdriver.Chrome(driver_path)

In [96]:
def get_reviews(url, delay=.5):

    # navigate to url
    driver.get(url)
    sleep(3)

    # get top reviews
    elem = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
    
    for i in range(5):
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(delay)

    sleep(3)
    
    for i in range(11):
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(delay)
    

    # save reviews
    page_content = driver.page_source   # grab html
    response = Selector(page_content)   # parse
    results = []                        # reviews storage list
    for el in response.xpath('//div/div[@data-review-id]/div[contains(@class, "content")]'):    # review xpath
        results.append({
            'title': el.xpath('.//div[contains(@class, "title")]/span/text()').extract_first(''),
            'rating': el.xpath('.//span[contains(@aria-label, "stars")]/@aria-label').extract_first('').replace('stars' ,'').strip(),
            'body': el.xpath('.//span[contains(@class, "text")]/text()').extract_first(''),
        })
        # save reviews loop source: https://outscraper.com/scraping-google-reviews-in-python/
    

    top_reviews = pd.DataFrame(results)

    # negative reviews
    driver.get(url) # reload driver
    sleep(3)

    # sort by worst reviews
    elem = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[7]/div[2]/button/span')
    elem.click()
    sleep(2)
    elem = driver.find_element_by_xpath('//*[@id="action-menu"]/ul/li[4]')
    elem.click()
    sleep(2)
    elem = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[7]/div[2]/button/span')

    # get negative reviews
    elem = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')

    for i in range(11):
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(delay)
    
    # save reviews
    page_content = driver.page_source   # grab html
    response = Selector(page_content)   # parse
    results = []                        # reviews storage list
    for el in response.xpath('//div/div[@data-review-id]/div[contains(@class, "content")]'):    # review xpath
        results.append({
            'title': el.xpath('.//div[contains(@class, "title")]/span/text()').extract_first(''),
            'rating': el.xpath('.//span[contains(@aria-label, "stars")]/@aria-label').extract_first('').replace('stars' ,'').strip(),
            'body': el.xpath('.//span[contains(@class, "text")]/text()').extract_first(''),
        })
        # save reviews loop source: https://outscraper.com/scraping-google-reviews-in-python/
    
    bottom_reivews = pd.DataFrame(results)

    # compile all reviews & return
    all_reviews = pd.concat([top_reviews, bottom_reivews], axis=0)
    all_reviews.reset_index(drop=True, inplace=True)
    all_reviews.drop_duplicates(inplace=True)

    return all_reviews

In [97]:
data = get_reviews(url)

In [88]:
driver.quit()

In [98]:
data

,title,rating,body
0,G Frye,5,The Central Park Zoo is a convenient location ...
1,Sarah Braxton,5,We went here on Christmas Eve day and it was m...
2,Donald Seigler,5,What a fun and quaint Zoo! We actually saw all...
3,Jason Cunningham,5,The zoo was a nice little visit. It was a litt...
4,Wanda Garrett,5,Beautiful ✨ park with a family-friendly atmosp...
5,Kateri Carpenter,4,"This zoo was fun, but it's not the best I've s..."
6,Vanesa Cg,5,I really enjoyed this zoo it was so much fun t...
7,Bernadette Bennett,4,Worth going for the seals! They are the main a...
8,Cesar Lopez,5,Make sure you check out the tropical exhibit. ...
9,Denisse H,5,Beautiful park & Family friendly! I had lots o...
